# Example

At this point we're overloading Jupyter notebook as our front end since I'm the only one using this app

In [1]:
from itertools import product
from functools import partial
import pandas as pd
import satori

In [2]:
try:
    df = pd.read_csv("./example/EUR=X-simpleCleaned.csv")
except:
    # example of data:
    df = pd.DataFrame({
        'High': [
            0.837240,
            0.837100,
            0.828020,
            0.830290,
            0.828780,], 
        'Low': [
            0.830560,
            0.825830,
            0.824400,
            0.823450,
            0.820280,],
        'Close': [
            0.835770,
            0.827200,
            0.824880,
            0.827750,
            0.820550,],})
past = df.iloc[:round(df.shape[0]*.8)]
future = df.iloc[round(df.shape[0]*.8):]

In [3]:
def getNewData() -> pd.DataFrame:
    ''' incrementally returns mock future data to simulate the passage of time '''
    for i in future.index:
        yield pd.DataFrame(future.loc[i]).T

In [4]:
data = satori.DataManager(
        data=past,
        getData=partial(next, getNewData()),
        validateData=satori.DataManager.defaultValidateData,
        appendData=satori.DataManager.defaultAppend)

In [5]:
def generateCombinedFeature(df:pd.DataFrame=None, columns:'list(str)'=None, prefix='Diff') -> pd.DataFrame:
    '''
    example of making a feature out of data you know ahead of time.
    most of the time you don't know what kinds of data you'll get...
    '''
    def name() -> str:
        return f'{prefix}{columns[0]}{columns[1]}'

    if df is None:
        return name()

    columns = columns or []
    feature = df.loc[:, columns[0]] - df.loc[:, columns[1]]
    feature.name = name()
    return feature

In [6]:
kwargs = {
    'hyperParameters': [
        satori.HyperParameter(
            name='n_estimators',
            value=300,
            kind=int,
            limit=100,
            minimum=200,
            maximum=5000),
        satori.HyperParameter(
            name='learning_rate',
            value=0.3,
            kind=float,
            limit=.05,
            minimum=.01,
            maximum=.1),
        satori.HyperParameter(
            name='max_depth',
            value=6,
            kind=int,
            limit=1,
            minimum=10,
            maximum=2),],
    'metrics':  {
        # raw data features
        'raw': satori.ModelManager.rawDataMetric,
        # daily percentage change, 1 day ago, 2 days ago, 3 days ago... 
        **{f'Daily{i}': partial(satori.ModelManager.dailyPercentChangeMetric, yesterday=i) for i in list(range(1, 31))},
        # rolling period transformation percentage change, max of the last 7 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(2, 21)))},
        # rolling period transformation percentage change, max of the last 50 or 70 days, etc... 
        **{f'Rolling{i}{tx[0:3]}': partial(satori.ModelManager.rollingPercentChangeMetric, window=i, transformation=tx)
            for tx, i in product('sum() max() min() mean() median() std()'.split(), list(range(22, 90, 7)))}},
    'features': {'DiffHighLow': partial(generateCombinedFeature, columns=['High', 'Low'])},
    'chosenFeatures': ['RawClose', 'RawHigh', 'RawLow', 'DiffHighLow'],
    'override': True}

In [7]:
modelHigh = satori.ModelManager(
    modelPath='modelHigh.joblib',
    targetKey='High',
    pinnedFeatures=['DiffHighLow'],
    **kwargs)

In [8]:
modelLow = satori.ModelManager(
    modelPath='modelLow.joblib',
    targetKey='Low',
    **kwargs)

In [9]:
modelClose = satori.ModelManager(
    modelPath='modelClose.joblib',
    targetKey='Close',
    **kwargs)

In [10]:
learner = satori.Engine(
    view=satori.JupyterView(points=8),
    data=data,
    models={modelHigh, modelLow, modelClose})

In [11]:
#learner.run()

In [12]:
learner.data.data

,High,Low,Close
0,0.83724,0.83056,0.83577
1,0.83710,0.82583,0.82720
2,0.82802,0.82440,0.82488
3,0.83029,0.82345,0.82775
4,0.82878,0.82028,0.82055
...,...,...,...
3671,0.80568,0.79885,0.79936
3672,0.80635,0.80140,0.80341
3673,0.81194,0.80460,0.80855
3674,0.81443,0.80624,0.80767


In [13]:
list(learner.models)[0].data

,High,Low,Close
0,0.83724,0.83056,0.83577
1,0.83710,0.82583,0.82720
2,0.82802,0.82440,0.82488
3,0.83029,0.82345,0.82775
4,0.82878,0.82028,0.82055
...,...,...,...
3671,0.80568,0.79885,0.79936
3672,0.80635,0.80140,0.80341
3673,0.81194,0.80460,0.80855
3674,0.81443,0.80624,0.80767


In [14]:
learner.data.runPublisher(learner.models)

In [15]:
import time
import threading
threads = {}

In [16]:
def subscriber():
    '''
    listens for external updates on subscriptions - 
    turn this into a steam rather than a loop - 
    triggered from flask app.
    this should probably be broken out into a service
    that subscribes and a service that listens...
    should be on demand
    '''
    while True:
        time.sleep(.1)
        learner.data.runSubscriber(learner.models)

In [17]:
def scholar():
    ''' always looks for external data and compiles it '''
    while True:
        learner.data.runScholar(learner.models)

In [18]:
threads['subscriber'] = threading.Thread(target=subscriber, daemon=True)

In [19]:
threads['scholar'] = threading.Thread(target=scholar, daemon=True)

In [20]:
def predictor(model:satori.ModelManager):
    ''' produces predictions on demand '''
    model.runPredictor(learner.data)

def sync(model:satori.ModelManager):
    ''' sync available inputs found and compiled by scholar on demand '''
    model.syncAvailableInputs(learner.data)

def explorer(model:satori.ModelManager):
    ''' always looks for a better model '''
    while True:
        model.runExplorer()

In [21]:
def watcher(model:satori.ModelManager):
    if learner.view:
        learner.view.listen(model)

In [22]:
for model in learner.models:
    model.buildStable() # we have to run this once for each model to complete its initialization
    print('built stable')

built stable
built stable
built stable


In [23]:
for model in self.models:
    watcher(model)
    print('wathing')

NameError: name 'self' is not defined

In [ ]:
for model in learner.models:
    predictor(model)
    print('ran predictor(model)', model.targetKey)    

In [ ]:
for model in learner.models:
    sync(model)
    print('ran sync(model)', model.targetKey)

In [ ]:
for model in learner.models:
    threads[f'{model.targetKey}.explorer'] = threading.Thread(target=explorer, args=[model], daemon=True)

In [ ]:
for thread in threads.values():
    thread.start()

print('started all threads')

In [ ]:
while threading.active_count() > 0:
    time.sleep(0)